In [39]:
import requests
import pandas as pd

from dotenv import load_dotenv
import os
load_dotenv()

gameName = "BakutoPhinn"
tagLine = "NA1"
api_key = os.environ.get("riot_api")

In [52]:
def get_puuid(gameName, tagLine, api_key):
    link = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}"
    response = requests.get(link)
    return response.json()['puuid']

def get_name_and_tag(puuid, api_key):
    link = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}"
    response = requests.get(link)
    return [response.json()['gameName'], response.json()['tagLine']]
# must use strings challengerleagues, grandmasterleagues, or masterleagues for league input
def get_league(league, api_key):
    link = f"https://na1.api.riotgames.com/lol/league/v4/{league}/by-queue/RANKED_SOLO_5x5?api_key={api_key}"
    return requests.get(link)

In [31]:
get_name_and_tag(get_puuid("crazy%20noche", "NA1", api_key), api_key)[0]

'Crazy Noche'

In [74]:
def get_ladder():
    chall_response = get_league("challengerleagues", api_key)
    chall_df = pd.DataFrame(chall_response.json()['entries'])
    chall_df = chall_df.sort_values("leaguePoints", ascending=False).reset_index(drop=True)
    gm_response = get_league("grandmasterleagues", api_key)
    gm_df = pd.DataFrame(gm_response.json()['entries'])
    gm_df = gm_df.sort_values("leaguePoints", ascending=False).reset_index(drop=True)
    masters_response = get_league("masterleagues", api_key)
    masters_df = pd.DataFrame(masters_response.json()['entries'])
    masters_df = masters_df.sort_values("leaguePoints", ascending=False).reset_index(drop=True)
    ladder = pd.concat([chall_df, gm_df, masters_df])
    ladder = ladder.drop(columns='rank')
    ladder = ladder.reset_index()
    ladder = ladder.rename(columns={"index":"Rank"})
    ladder["Rank"] = ladder["Rank"]+1
    return ladder